### Cluster event Summary Generation using OpenAI API

#### Load the Clustered Keywords Data

In [ ]:
# Read the data if necessary
keys_dff = pd.read_pickle("Final_clustered_data.pkl")
#keys_dff = pd.read_pickle("Final_clustered_data.csv")
keys_dff.columns

In [ ]:
import pandas as pd

# Check the output
print(keys_dff.shape)
keys_dff.head()

In [218]:
import os
import openai

# Set your OpenAI API key
openai.api_key = "key"
#  access the API key through os.environ
os.environ['OPENAI_API_KEY'] = openai.api_key

In [219]:
def call_openai_api(messages):
    return openai.ChatCompletion.create(
        model='gpt-3.5-turbo-16k',
        messages=messages,
        max_tokens=4096,
        temperature=1
    )

def summarize_text(transcript):
    system_prompt = "You are a news summarizer and know all global news. You're helping me write a concise summary of the given news text "
    user_prompt = f"""Please summarize the text provided below in 10 words, capturing the key points concisely and accurately.
     Do not include any redundant phrases or unnecessary details.
     Do not include words like "The text contains...", or "The text includes..."
     Text: {text}
    """
    
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]
    
    response = call_openai_api(messages)
    summary = response['choices'][0]['message']['content']
    return summary

In [ ]:
# prevent rate limits imposed by the GPT modelsimport time
df1=pd.DataFrame()
for i in range(8): # for 8000 rows in dataset
    init = i*100
    final = (i+1)*100
    csum = keys_dff.iloc[init:final]
    print(i, init, final)
    csum['summary_llm'] = csum['Text'].map(lambda x: summarize_text(x))
    df1 = pd.concat([df1, csum])
    time.sleep(60) 

In [ ]:
print(df1.shape)
df1.shape

In [ ]:
# Save embeddings and clusters with openAI summary for each cluster content 
df1.to_pickle("Final_clustered_data_summary.pkl")
df1.to_csv("Final_clustered_data_summary.csv")
df1.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
df1.head(10)